In [144]:
import numpy as np
import random
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import time
import scipy
from scipy.spatial import distance_matrix


In [2]:
def schaffer_function(mat_x_y):
    x = mat_x_y[:, 0]
    y = mat_x_y[:, 1]
    g = 0.5 + (np.power((np.sin( np.sqrt( np.power(x, 2) + np.power(y, 2)))), 2) - 0.5)/ \
        (1 + 0.001 * (np.power(x, 2) + np.power(y, 2)))
    return g

In [190]:
class OptAiNet():
    def __init__(self,
                 num_epochs:int,
                 pop_size:int,
                 Nc:int,
                 chrom_length:int,
                 value_ranges:list,
                 fitness_func, # Function Type,
                 beta=100,
                 clone_threshold = 0.01,
                 supression_threshold=0.2,
                 newcomers_percentage = 0.4,
                 seed=42,
                 eval_every=100,
                 verbose = 0,
                ):
        
        self.num_epochs = num_epochs
        self.pop_size = pop_size
        self.value_ranges = np.array(value_ranges)
        self.fitness_func = fitness_func
        self.chrom_length = chrom_length
        self.Nc = Nc
        self.beta = beta
        self.clone_threshold = clone_threshold
        self.supression_threshold = supression_threshold
        self.newcomers_percentage = newcomers_percentage

        self.f_avg_clone_mutated_previous = 0
        self.continue_clone = True

        self.seed = seed    
        self.best_ind_list = np.zeros(self.num_epochs)
        self.avg_ind_list = np.zeros(self.num_epochs)
        self.eval_every = eval_every
        self.verbose = verbose
        np.random.seed(seed=seed)

        self.init_pop()
        self.clone()
        self.mutation()
        self.evaluation()
        self.calculate_affinity()
    
    def init_pop(self):
        self.pop = np.random.rand(self.pop_size, self.chrom_length)
        self.min_mat = self.value_ranges.T[0, :]
        self.max_mat = self.value_ranges.T[1,:]
        self.pop = self.pop * (self.max_mat - self.min_mat) + self.min_mat
        self.f_pop = self.fitness_func(self.pop)
        self.f_pop_clone = self.f_pop.copy()
        self.pop_clone = self.pop.copy()
        #print(self.f_pop)
    
    def clone(self):
        self.pop_clone = np.repeat(self.pop_clone, repeats=self.Nc, axis=0)
        #self.pop_clone = self.pop_clone.reshape((1, self.Nc, -1, self.chrom_length))
        #self.formatted_pop_clone = self.pop_clone.reshape((-1, self.chrom_length))
        self.f_pop_clone = self.fitness_func(self.pop_clone)
        f_max = self.f_pop_clone.max()
        f_min = self.f_pop_clone.min()
        self.f_pop_clone_norm = (self.f_pop_clone - f_min)/ (f_max - f_min)

    def mutation(self):
        self.alpha = (1/self.beta) * np.exp(self.f_pop_clone_norm)
        self.alpha = np.repeat(self.alpha, self.chrom_length)
        self.random_mutation = np.random.normal(0, 1, size=self.pop_clone.shape[0] * self.pop_clone.shape[1])
        self.random_mutation = self.random_mutation.reshape(self.pop_clone.shape[0], self.pop_clone.shape[1])
        self.alpha = self.alpha.reshape(self.random_mutation.shape)
        self.pop_clone_mutated = self.pop_clone + self.alpha * self.random_mutation

        # Stops algorithm from violating the delimited search area
        self.repeat_min_mat = np.tile(self.min_mat, (self.pop_clone_mutated.shape[0], 1))
        self.repeat_max_mat = np.tile(self.max_mat, (self.pop_clone_mutated.shape[0], 1))
        self.pop_clone_mutated[self.pop_clone_mutated < self.repeat_min_mat] = self.repeat_min_mat[self.pop_clone_mutated < self.repeat_min_mat]
        self.pop_clone_mutated[self.pop_clone_mutated > self.repeat_max_mat] = self.repeat_max_mat[self.pop_clone_mutated > self.repeat_max_mat]

        self.f_pop_clone_mutated = self.fitness_func(self.pop_clone_mutated)
        self.pop_clone = self.pop_clone_mutated
        self.f_pop_clone = self.f_pop_clone_mutated
    
    def evaluation(self):
        self.f_avg_clone_mutated = self.f_pop_clone_mutated.mean()
        if np.abs(self.f_avg_clone_mutated - self.f_avg_clone_mutated_previous) < self.clone_threshold:
            self.continue_clone = False
        else:
            self.continue_clone = True
        self.f_avg_clone_mutated_previous = self.f_avg_clone_mutated
    
    def calculate_affinity(self):
        # Calculates affinity (euclidian distance between all points)
        distances = distance_matrix(self.pop_clone, self.pop_clone)
        # Replicates the fitness results into a matrix pop_clone_size x pop_clone_size
        f_pop_clone_matrix = np.tile(self.f_pop_clone, (distances.shape[0], 1))        
        # Uses a mask to find which values are lower than the threshold
        masked_f = f_pop_clone_matrix * (distances<self.supression_threshold)
        best_indices = np.where(masked_f == masked_f.max(axis=1).T)[1]
        best_indices = np.unique(best_indices)
        self.memory_cells = self.pop_clone[best_indices]
        print(best_indices)
        print(self.pop_clone)
        print(self.f_pop_clone)
        print(self.memory_cells)


opt_ai_net = OptAiNet( 
                num_epochs=100,
                pop_size=2,
                Nc=2,
                chrom_length=2,
                value_ranges=[(-10,10), (-10,10)],
                fitness_func=schaffer_function
            )

[0 3]
[[-2.51153916  9.01194476]
 [-2.49340549  9.02196048]
 [ 4.6271172   1.987918  ]
 [ 4.62728184  1.96050984]]
[0.04466205 0.04410981 0.8889718  0.89516212]
[[-2.51153916  9.01194476]
 [ 4.62728184  1.96050984]]
